# Module 2: Structured Data Modeling and Import

Welcome to Module 2! Now that you understand graph fundamentals, let's dive into the practical world of data modeling and importing real datasets into Neo4j. This is where your graph applications start to come alive with meaningful data.

## 🎯 What You'll Learn

By the end of this module, you'll be able to:
- Design effective graph data models for complex business scenarios
- Import CSV and structured data efficiently into Neo4j
- Apply data modeling best practices and design patterns
- Handle data quality issues and validation
- Optimize import performance for large datasets
- Create indexes and constraints for data integrity

## 🧠 Why This Matters

Most real-world graph applications start with existing data in spreadsheets, databases, or APIs. Learning to effectively model and import this data is crucial because:

**Poor data modeling** leads to:
- Complex, slow queries
- Difficult maintenance
- Limited scalability

**Good data modeling** enables:
- Intuitive, fast queries
- Easy application development
- Excellent performance at scale

---

## 📚 Prerequisites

- Completion of Module 1: Graph Basics
- Understanding of basic Cypher queries
- Familiarity with CSV data and spreadsheets

---

# Lesson 1: Data Modeling Fundamentals (15 minutes)

## 🤔 Think About This Real-World Scenario

Imagine you're building a movie recommendation system like Netflix. You have spreadsheets with:
- Movies (titles, genres, release dates, budgets)
- People (actors, directors, writers)
- User ratings and reviews
- Box office data

**Question**: How do you transform this tabular data into a graph that can power intelligent recommendations?

This is the challenge of **data modeling** - designing a graph structure that represents your domain effectively.

## 🏗️ The Data Modeling Process

Graph data modeling follows a proven process:

### 1. **Understand the Domain**
- What questions will users ask?
- What are the key entities and how do they relate?
- What business rules apply?

### 2. **Identify the Entities** (Nodes)
- **Nouns** in your domain → Nodes
- Examples: Movie, Person, Genre, User

### 3. **Identify the Relationships**
- **Verbs** describing connections → Relationships
- Examples: ACTED_IN, DIRECTED, RATED, SIMILAR_TO

### 4. **Add Properties**
- **Attributes** of entities and relationships → Properties
- Examples: title, name, rating, timestamp

### 5. **Validate with Queries**
- Can you answer the key business questions?
- Are the queries intuitive and performant?

## 💡 Data Modeling Principles

### ✅ Good Practices:
- **Model for your queries**: Design around the questions you need to answer
- **Use meaningful names**: `ACTED_IN` not `REL_TYPE_1`
- **Represent relationships explicitly**: Don't hide connections in properties
- **Favor specific relationships**: `DIRECTED` vs generic `ASSOCIATED_WITH`

### ❌ Common Mistakes:
- **Modeling like SQL**: Thinking in tables instead of connections
- **Over-normalization**: Creating unnecessary intermediate nodes
- **Under-normalization**: Storing everything as properties
- **Generic relationships**: Losing semantic meaning

## 🚀 Let's Start With Setup

First, let's set up our environment for this module:

In [ ]:
# Install required libraries for data processing and Neo4j
!pip install neo4j pandas numpy requests faker

print("✅ Libraries installed! Ready for data modeling and import.")

In [ ]:
# Import our toolkit for data modeling
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime, date
import requests
from faker import Faker
import random

# Set up faker for generating realistic data
fake = Faker()
random.seed(42)  # For reproducible examples

print("📚 Data modeling toolkit ready!")

In [ ]:
# Neo4j connection setup
NEO4J_URI = os.getenv('NEO4J_URI', 'bolt://localhost:7687')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME', 'neo4j')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD', 'password')

# Create our connection
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

def run_query(query, parameters=None):
    """
    Execute a Cypher query and return results.
    This helper function makes it easy to run queries throughout the module.
    """
    with driver.session() as session:
        result = session.run(query, parameters or {})
        return [record.data() for record in result]

# Test connection
test_result = run_query("RETURN 'Ready for data modeling!' as message")
print(f"🎉 {test_result[0]['message']}")

## 🎬 Our Domain: Movie Recommendation System

Let's model a movie recommendation system similar to what Netflix or IMDb might use. This domain is rich with relationships and perfect for demonstrating graph modeling principles.

### Key Questions Our Model Should Answer:
1. "What movies did Tom Hanks star in?"
2. "Who directed Inception?"
3. "What genres does this user prefer?"
4. "Which movies are similar to ones I've rated highly?"
5. "What movies should I recommend to this user?"

### Domain Analysis:

**Entities (Nodes):**
- `Movie` - Films in our database
- `Person` - Actors, directors, writers
- `Genre` - Categories like Action, Drama, Comedy
- `User` - People who rate and review movies

**Relationships:**
- `ACTED_IN` - Person → Movie
- `DIRECTED` - Person → Movie  
- `WROTE` - Person → Movie
- `IN_GENRE` - Movie → Genre
- `RATED` - User → Movie
- `SIMILAR_TO` - Movie → Movie

Let's start by clearing our workspace and building this model step by step:

In [ ]:
# Clear the database for our movie modeling exercise
run_query("MATCH (n) DETACH DELETE n")
print("🧹 Database cleared - ready to build our movie recommendation graph!")

# Let's also check that we have a clean start
node_count = run_query("MATCH (n) RETURN count(n) as count")[0]['count']
print(f"Current node count: {node_count} (should be 0)")

### Step 1: Create Our Schema - Constraints and Indexes

Before importing data, it's a best practice to set up constraints and indexes. This ensures data quality and query performance:

In [ ]:
# Create uniqueness constraints to prevent duplicate data
# These also automatically create indexes for performance

constraints = [
    "CREATE CONSTRAINT movie_id IF NOT EXISTS FOR (m:Movie) REQUIRE m.id IS UNIQUE",
    "CREATE CONSTRAINT person_id IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE", 
    "CREATE CONSTRAINT genre_name IF NOT EXISTS FOR (g:Genre) REQUIRE g.name IS UNIQUE",
    "CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE u.id IS UNIQUE"
]

print("🔒 Creating constraints for data integrity...")
for constraint in constraints:
    try:
        run_query(constraint)
        print(f"   ✅ {constraint.split()[2]} constraint created")
    except Exception as e:
        if "already exists" in str(e):
            print(f"   ⚠️ {constraint.split()[2]} constraint already exists")
        else:
            print(f"   ❌ Error creating constraint: {e}")

print("\n🚀 Schema setup complete! Our data will be clean and performant.")

## 🎯 Knowledge Check #1

Before we continue, let's make sure you understand the modeling decisions:

**Question**: Why did we create separate `Person` and `Movie` nodes instead of storing actor names as properties on movies?

**Think about it, then check below:**

<details>
<summary>Click to see the answer</summary>

**Separate nodes are better because:**

1. **Reusability**: Tom Hanks appears in many movies - we store his info once, not repeatedly
2. **Relationships**: We can query "all movies Tom Hanks starred in" naturally
3. **Rich data**: People have their own properties (birthdate, nationality, etc.)
4. **Query power**: "Find actors who worked together" becomes a simple graph traversal
5. **Data integrity**: Changes to person info update everywhere automatically

This is the power of **thinking in graphs** vs. thinking in tables!

</details>

---

# Lesson 2: Creating Sample Data (15 minutes)

## 📊 Generating Realistic Movie Data

To understand data modeling and import patterns, let's create realistic sample data. In real projects, this data would come from APIs, databases, or CSV files.

### Creating Our Dataset

We'll generate data that mimics what you'd find in real movie databases:

In [ ]:
# Generate realistic movie data
def create_movie_dataset():
    """Generate a realistic movie dataset for our modeling exercise"""
    
    print("🎬 Generating movie dataset...")
    
    # Define genres
    genres = [
        'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
        'Drama', 'Family', 'Fantasy', 'Horror', 'Mystery', 'Romance', 
        'Science Fiction', 'Thriller', 'War', 'Western'
    ]
    
    # Generate people (actors, directors, writers)
    people = []
    for i in range(50):
        person = {
            'id': f'person_{i:03d}',
            'name': fake.name(),
            'born': fake.date_between(start_date='-80y', end_date='-20y'),
            'nationality': fake.country(),
            'roles': random.sample(['actor', 'director', 'writer'], k=random.randint(1, 2))
        }
        people.append(person)
    
    # Generate movies
    movies = []
    for i in range(30):
        # Create realistic movie titles
        title_patterns = [
            f"The {fake.word().title()}",
            f"{fake.word().title()} {fake.word().title()}",
            f"{fake.first_name()}'s {fake.word().title()}",
            f"The {fake.word().title()} of {fake.word().title()}"
        ]
        
        movie = {
            'id': f'movie_{i:03d}',
            'title': random.choice(title_patterns),
            'released': fake.date_between(start_date='-30y', end_date='today'),
            'runtime': random.randint(90, 180),
            'budget': random.randint(1000000, 200000000),
            'revenue': random.randint(500000, 500000000),
            'plot': fake.text(max_nb_chars=200),
            'genres': random.sample(genres, k=random.randint(1, 3))
        }
        movies.append(movie)
    
    # Generate users
    users = []
    for i in range(20):
        user = {
            'id': f'user_{i:03d}',
            'username': fake.user_name(),
            'email': fake.email(),
            'joined': fake.date_between(start_date='-5y', end_date='today'),
            'age': random.randint(18, 70)
        }
        users.append(user)
    
    return {
        'people': people,
        'movies': movies,
        'genres': [{'name': genre} for genre in genres],
        'users': users
    }

# Generate our dataset
dataset = create_movie_dataset()

print(f"📊 Dataset created:")
print(f"   - {len(dataset['people'])} people")
print(f"   - {len(dataset['movies'])} movies")
print(f"   - {len(dataset['genres'])} genres")
print(f"   - {len(dataset['users'])} users")

# Show sample data
print(f"\n🎬 Sample movie: {dataset['movies'][0]['title']}")
print(f"   Released: {dataset['movies'][0]['released']}")
print(f"   Genres: {', '.join(dataset['movies'][0]['genres'])}")
print(f"   Plot: {dataset['movies'][0]['plot'][:100]}...")

### Understanding Our Data Structure

Before importing, let's examine what we have. This step is crucial in real projects - you need to understand your data before modeling it:

In [ ]:
# Let's explore our data structure
print("🔍 Exploring our dataset structure...\n")

# Convert to pandas for easy analysis
movies_df = pd.DataFrame(dataset['movies'])
people_df = pd.DataFrame(dataset['people'])
users_df = pd.DataFrame(dataset['users'])

print("📽️ Movies DataFrame info:")
print(f"   Shape: {movies_df.shape}")
print(f"   Columns: {list(movies_df.columns)}")
print(f"   Date range: {movies_df['released'].min()} to {movies_df['released'].max()}")
print(f"   Budget range: ${movies_df['budget'].min():,} to ${movies_df['budget'].max():,}")

print("\n👥 People DataFrame info:")
print(f"   Shape: {people_df.shape}")
print(f"   Columns: {list(people_df.columns)}")

# Show role distribution
all_roles = []
for person in dataset['people']:
    all_roles.extend(person['roles'])
role_counts = pd.Series(all_roles).value_counts()
print(f"   Role distribution: {dict(role_counts)}")

print("\n👤 Users DataFrame info:")
print(f"   Shape: {users_df.shape}")
print(f"   Age range: {users_df['age'].min()} to {users_df['age'].max()}")

# Display sample records
print("\n📊 Sample records:")
print("\nSample Movie:")
print(json.dumps(dataset['movies'][0], indent=2, default=str))

print("\nSample Person:")
print(json.dumps(dataset['people'][0], indent=2, default=str))

## 💾 Save Data for Import Practice

In real projects, you often receive data as CSV files. Let's save our data in CSV format to practice import techniques:

In [ ]:
# Save our datasets as CSV files for import practice
import os

# Create data directory
data_dir = 'movie_data'
os.makedirs(data_dir, exist_ok=True)

print("💾 Saving datasets as CSV files...")

# Save movies (need to handle list columns)
movies_for_csv = []
for movie in dataset['movies']:
    movie_copy = movie.copy()
    movie_copy['genres'] = '|'.join(movie['genres'])  # Convert list to pipe-separated string
    movies_for_csv.append(movie_copy)

movies_df = pd.DataFrame(movies_for_csv)
movies_df.to_csv(f'{data_dir}/movies.csv', index=False)
print(f"   ✅ movies.csv ({len(movies_df)} records)")

# Save people (handle roles list)
people_for_csv = []
for person in dataset['people']:
    person_copy = person.copy()
    person_copy['roles'] = '|'.join(person['roles'])
    people_for_csv.append(person_copy)

people_df = pd.DataFrame(people_for_csv)
people_df.to_csv(f'{data_dir}/people.csv', index=False)
print(f"   ✅ people.csv ({len(people_df)} records)")

# Save genres
genres_df = pd.DataFrame(dataset['genres'])
genres_df.to_csv(f'{data_dir}/genres.csv', index=False)
print(f"   ✅ genres.csv ({len(genres_df)} records)")

# Save users
users_df = pd.DataFrame(dataset['users'])
users_df.to_csv(f'{data_dir}/users.csv', index=False)
print(f"   ✅ users.csv ({len(users_df)} records)")

print(f"\n📁 All CSV files saved to '{data_dir}/' directory")

# Show what our CSV looks like
print("\n👀 Sample CSV content (movies.csv):")
print(movies_df.head(3).to_string(index=False))

## 🎯 Knowledge Check #2

**Question**: Why did we convert the `genres` list to a pipe-separated string when saving to CSV?

**Think about it, then check below:**

<details>
<summary>Click to see the answer</summary>

**CSV limitations require this transformation:**

1. **CSV format**: CSV files can't natively store arrays/lists - each cell contains one value
2. **Common pattern**: Pipe-separated (or comma-separated) strings are a standard way to represent multiple values in CSV
3. **Import flexibility**: When importing, we can easily split the string back into individual genre relationships
4. **Readability**: Humans can still read "Action|Adventure|Drama" in the CSV file

This is a common data engineering pattern you'll use frequently when working with real datasets!

</details>

---

# Lesson 3: Data Import Strategies (20 minutes)

## 🚀 The Art of Data Import

Now for the exciting part - bringing our data to life in Neo4j! There are several strategies for importing data, each with its own advantages.

### Import Strategy Options:

1. **CREATE statements** - Good for small datasets, learning
2. **LOAD CSV** - Built-in, handles medium datasets well
3. **APOC procedures** - Advanced features, large datasets
4. **Bulk import tool** - Massive datasets, initial loads
5. **Application code** - Custom logic, real-time imports

We'll focus on the first two since they cover most use cases.

## Method 1: Direct CREATE Statements

Let's start by importing our data using direct CREATE statements. This approach gives you full control and is perfect for understanding the process:

In [ ]:
# Method 1: Import using direct CREATE statements
print("🏗️ Importing data using CREATE statements...\n")

# Step 1: Import Genres (they have no dependencies)
print("1️⃣ Creating genres...")
for genre in dataset['genres']:
    run_query("""
    CREATE (g:Genre {name: $name})
    """, {'name': genre['name']})

genre_count = run_query("MATCH (g:Genre) RETURN count(g) as count")[0]['count']
print(f"   ✅ Created {genre_count} genres")

# Step 2: Import People
print("\n2️⃣ Creating people...")
for person in dataset['people']:
    run_query("""
    CREATE (p:Person {
        id: $id,
        name: $name,
        born: date($born),
        nationality: $nationality
    })
    """, {
        'id': person['id'],
        'name': person['name'],
        'born': person['born'].isoformat(),
        'nationality': person['nationality']
    })

people_count = run_query("MATCH (p:Person) RETURN count(p) as count")[0]['count']
print(f"   ✅ Created {people_count} people")

# Step 3: Import Users
print("\n3️⃣ Creating users...")
for user in dataset['users']:
    run_query("""
    CREATE (u:User {
        id: $id,
        username: $username,
        email: $email,
        joined: date($joined),
        age: $age
    })
    """, {
        'id': user['id'],
        'username': user['username'],
        'email': user['email'],
        'joined': user['joined'].isoformat(),
        'age': user['age']
    })

user_count = run_query("MATCH (u:User) RETURN count(u) as count")[0]['count']
print(f"   ✅ Created {user_count} users")

# Step 4: Import Movies and create genre relationships
print("\n4️⃣ Creating movies and connecting to genres...")
for movie in dataset['movies']:
    # Create the movie
    run_query("""
    CREATE (m:Movie {
        id: $id,
        title: $title,
        released: date($released),
        runtime: $runtime,
        budget: $budget,
        revenue: $revenue,
        plot: $plot
    })
    """, {
        'id': movie['id'],
        'title': movie['title'],
        'released': movie['released'].isoformat(),
        'runtime': movie['runtime'],
        'budget': movie['budget'],
        'revenue': movie['revenue'],
        'plot': movie['plot']
    })
    
    # Connect to genres
    for genre_name in movie['genres']:
        run_query("""
        MATCH (m:Movie {id: $movie_id}),
              (g:Genre {name: $genre_name})
        CREATE (m)-[:IN_GENRE]->(g)
        """, {
            'movie_id': movie['id'],
            'genre_name': genre_name
        })

movie_count = run_query("MATCH (m:Movie) RETURN count(m) as count")[0]['count']
genre_rel_count = run_query("MATCH ()-[r:IN_GENRE]->() RETURN count(r) as count")[0]['count']
print(f"   ✅ Created {movie_count} movies")
print(f"   ✅ Created {genre_rel_count} genre relationships")

print("\n🎉 Data import complete using CREATE statements!")

### Create Movie Industry Relationships

Now let's add the relationships that make our graph interesting - who acted in what, who directed what, etc.:

In [ ]:
# Create realistic movie industry relationships
print("🎬 Creating movie industry relationships...\n")

# Get our data for creating relationships
movies = run_query("MATCH (m:Movie) RETURN m.id as id")
people = run_query("MATCH (p:Person) RETURN p.id as id")

# Create ACTED_IN relationships
print("1️⃣ Creating ACTED_IN relationships...")
acted_count = 0
for movie in movies:
    # Each movie has 3-6 actors
    num_actors = random.randint(3, 6)
    selected_actors = random.sample(people, num_actors)
    
    for actor in selected_actors:
        # Create relationship with role property
        role = fake.first_name() if random.random() > 0.7 else None  # Some have character names
        run_query("""
        MATCH (p:Person {id: $person_id}),
              (m:Movie {id: $movie_id})
        CREATE (p)-[:ACTED_IN {role: $role}]->(m)
        """, {
            'person_id': actor['id'],
            'movie_id': movie['id'],
            'role': role
        })
        acted_count += 1

print(f"   ✅ Created {acted_count} ACTED_IN relationships")

# Create DIRECTED relationships
print("\n2️⃣ Creating DIRECTED relationships...")
directed_count = 0
for movie in movies:
    # Each movie has 1-2 directors
    num_directors = random.randint(1, 2)
    selected_directors = random.sample(people, num_directors)
    
    for director in selected_directors:
        run_query("""
        MATCH (p:Person {id: $person_id}),
              (m:Movie {id: $movie_id})
        CREATE (p)-[:DIRECTED]->(m)
        """, {
            'person_id': director['id'],
            'movie_id': movie['id']
        })
        directed_count += 1

print(f"   ✅ Created {directed_count} DIRECTED relationships")

# Create WROTE relationships
print("\n3️⃣ Creating WROTE relationships...")
wrote_count = 0
for movie in movies:
    # Each movie has 1-3 writers
    num_writers = random.randint(1, 3)
    selected_writers = random.sample(people, num_writers)
    
    for writer in selected_writers:
        run_query("""
        MATCH (p:Person {id: $person_id}),
              (m:Movie {id: $movie_id})
        CREATE (p)-[:WROTE]->(m)
        """, {
            'person_id': writer['id'],
            'movie_id': movie['id']
        })
        wrote_count += 1

print(f"   ✅ Created {wrote_count} WROTE relationships")

print("\n🎭 Movie industry relationships created!")
print(f"Total relationships: {acted_count + directed_count + wrote_count}")

### Add User Ratings

Finally, let's add user ratings to make our recommendation system complete:

In [ ]:
# Create user ratings
print("⭐ Creating user ratings...\n")

users = run_query("MATCH (u:User) RETURN u.id as id")
movies = run_query("MATCH (m:Movie) RETURN m.id as id")

rating_count = 0
for user in users:
    # Each user rates 5-15 movies
    num_ratings = random.randint(5, 15)
    selected_movies = random.sample(movies, num_ratings)
    
    for movie in selected_movies:
        # Generate realistic ratings (skewed toward positive)
        rating = random.choices(
            [1, 2, 3, 4, 5],
            weights=[5, 10, 20, 35, 30]  # More positive ratings
        )[0]
        
        # Create rating relationship
        run_query("""
        MATCH (u:User {id: $user_id}),
              (m:Movie {id: $movie_id})
        CREATE (u)-[:RATED {
            rating: $rating,
            timestamp: datetime($timestamp)
        }]->(m)
        """, {
            'user_id': user['id'],
            'movie_id': movie['id'],
            'rating': rating,
            'timestamp': fake.date_time_between(start_date='-2y', end_date='now').isoformat()
        })
        rating_count += 1

print(f"   ✅ Created {rating_count} user ratings")

# Let's see the rating distribution
rating_dist = run_query("""
MATCH ()-[r:RATED]->()
RETURN r.rating as rating, count(*) as count
ORDER BY rating
""")

print("\n📊 Rating distribution:")
for dist in rating_dist:
    stars = '⭐' * int(dist['rating'])
    print(f"   {stars} ({dist['rating']}): {dist['count']} ratings")

print("\n🎉 Complete movie recommendation graph created!")

## 📊 Verify Our Import

Let's verify that our import worked correctly and explore what we've created:

In [ ]:
# Comprehensive verification of our imported data
print("🔍 Verifying our movie recommendation graph...\n")

# Node counts
node_counts = run_query("""
MATCH (n)
RETURN labels(n)[0] as label, count(n) as count
ORDER BY count DESC
""")

print("📊 Node counts:")
total_nodes = 0
for count_info in node_counts:
    print(f"   {count_info['label']}: {count_info['count']:,}")
    total_nodes += count_info['count']
print(f"   TOTAL: {total_nodes:,} nodes")

# Relationship counts
rel_counts = run_query("""
MATCH ()-[r]-()
RETURN type(r) as relationship_type, count(r) as count
ORDER BY count DESC
""")

print("\n🔗 Relationship counts:")
total_rels = 0
for rel_info in rel_counts:
    print(f"   {rel_info['relationship_type']}: {rel_info['count']:,}")
    total_rels += rel_info['count']
print(f"   TOTAL: {total_rels:,} relationships")

# Sample queries to verify data quality
print("\n✅ Data quality checks:")

# Check 1: Movies with genres
movies_with_genres = run_query("""
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
RETURN count(DISTINCT m) as movies_with_genres
""")[0]['movies_with_genres']
total_movies = run_query("MATCH (m:Movie) RETURN count(m) as count")[0]['count']
print(f"   Movies with genres: {movies_with_genres}/{total_movies} ({movies_with_genres/total_movies:.1%})")

# Check 2: Movies with cast
movies_with_cast = run_query("""
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
RETURN count(DISTINCT m) as movies_with_cast
""")[0]['movies_with_cast']
print(f"   Movies with cast: {movies_with_cast}/{total_movies} ({movies_with_cast/total_movies:.1%})")

# Check 3: Users with ratings
users_with_ratings = run_query("""
MATCH (u:User)-[:RATED]->(m:Movie)
RETURN count(DISTINCT u) as users_with_ratings
""")[0]['users_with_ratings']
total_users = run_query("MATCH (u:User) RETURN count(u) as count")[0]['count']
print(f"   Users with ratings: {users_with_ratings}/{total_users} ({users_with_ratings/total_users:.1%})")

print("\n🎉 Graph verification complete - data looks good!")

## 🎯 Test Our Model with Business Queries

The true test of a good data model is whether it can answer business questions elegantly. Let's try the questions we identified earlier:

In [ ]:
# Test our data model with real business queries
print("🎯 Testing our model with business questions...\n")

# Question 1: "What movies did [person] star in?"
sample_actor = run_query("MATCH (p:Person)-[:ACTED_IN]->() RETURN p.name as name LIMIT 1")[0]['name']
actor_movies = run_query("""
MATCH (p:Person {name: $name})-[:ACTED_IN]->(m:Movie)
RETURN m.title as title, m.released as released
ORDER BY m.released DESC
""", {'name': sample_actor})

print(f"1️⃣ Movies starring {sample_actor}:")
for movie in actor_movies[:3]:
    print(f"   - {movie['title']} ({movie['released'].year})")

# Question 2: "Who directed [movie]?"
sample_movie = run_query("MATCH (m:Movie)<-[:DIRECTED]-() RETURN m.title as title LIMIT 1")[0]['title']
directors = run_query("""
MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: $title})
RETURN p.name as director
""", {'title': sample_movie})

print(f"\n2️⃣ Directors of '{sample_movie}':")
for director in directors:
    print(f"   - {director['director']}")

# Question 3: "What genres does this user prefer?"
sample_user = run_query("MATCH (u:User)-[:RATED]->() RETURN u.username as username LIMIT 1")[0]['username']
user_genres = run_query("""
MATCH (u:User {username: $username})-[r:RATED]->(m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE r.rating >= 4
RETURN g.name as genre, count(*) as high_rated_count
ORDER BY high_rated_count DESC
LIMIT 3
""", {'username': sample_user})

print(f"\n3️⃣ Preferred genres for user '{sample_user}' (4+ star ratings):")
for genre in user_genres:
    print(f"   - {genre['genre']}: {genre['high_rated_count']} movies")

# Question 4: "Which movies are similar to ones I've rated highly?"
similar_movies = run_query("""
MATCH (u:User {username: $username})-[r:RATED]->(liked:Movie)
WHERE r.rating >= 4
MATCH (liked)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(similar:Movie)
WHERE NOT (u)-[:RATED]->(similar)
WITH similar, count(g) as shared_genres
RETURN similar.title as title, shared_genres
ORDER BY shared_genres DESC
LIMIT 3
""", {'username': sample_user})

print(f"\n4️⃣ Movies similar to ones {sample_user} liked:")
for movie in similar_movies:
    print(f"   - {movie['title']} ({movie['shared_genres']} shared genres)")

# Question 5: "What movies should I recommend to this user?"
recommendations = run_query("""
MATCH (target:User {username: $username})-[r1:RATED]->(m:Movie)<-[r2:RATED]-(other:User)
WHERE r1.rating >= 4 AND r2.rating >= 4 AND target <> other
MATCH (other)-[r3:RATED]->(rec:Movie)
WHERE r3.rating >= 4 AND NOT (target)-[:RATED]->(rec)
RETURN rec.title as recommended_movie, 
       count(*) as recommendation_strength,
       avg(r3.rating) as avg_rating
ORDER BY recommendation_strength DESC, avg_rating DESC
LIMIT 3
""", {'username': sample_user})

print(f"\n5️⃣ Recommendations for {sample_user} (collaborative filtering):")
for rec in recommendations:
    print(f"   - {rec['recommended_movie']} (strength: {rec['recommendation_strength']}, avg rating: {rec['avg_rating']:.1f})")

print("\n✅ All business questions answered successfully!")
print("🎉 Our data model is working perfectly for movie recommendations!")

## 🎯 Knowledge Check #3

**Question**: Looking at our recommendation query (#5), explain why it works and what pattern it's using.

**Think about it, then check below:**

<details>
<summary>Click to see the answer</summary>

**This is collaborative filtering in action:**

1. **Find similar users**: Users who rated the same movies highly as the target user
2. **Discover their preferences**: What other movies did these similar users rate highly?
3. **Filter unrated**: Only recommend movies the target user hasn't seen
4. **Rank by strength**: Movies recommended by multiple similar users rank higher

**The graph pattern**: `(target)-[:RATED]->(movie)<-[:RATED]-(similar)-[:RATED]->(recommendation)`

This is incredibly natural in a graph but would require complex JOINs in SQL. The graph structure makes collaborative filtering almost trivial to implement!

</details>

---

# Lesson 4: Advanced Import Techniques (15 minutes)

## 🚀 Method 2: LOAD CSV - The Production Approach

While CREATE statements are great for learning, `LOAD CSV` is the go-to method for production imports. It's built into Neo4j and handles larger datasets efficiently.

Let's clear our database and reimport using LOAD CSV to see the difference:

In [ ]:
# Clear database for LOAD CSV demonstration
print("🧹 Clearing database for LOAD CSV demonstration...")
run_query("MATCH (n) DETACH DELETE n")

# Recreate constraints (they were deleted with the data)
constraints = [
    "CREATE CONSTRAINT movie_id IF NOT EXISTS FOR (m:Movie) REQUIRE m.id IS UNIQUE",
    "CREATE CONSTRAINT person_id IF NOT EXISTS FOR (p:Person) REQUIRE p.id IS UNIQUE", 
    "CREATE CONSTRAINT genre_name IF NOT EXISTS FOR (g:Genre) REQUIRE g.name IS UNIQUE",
    "CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE u.id IS UNIQUE"
]

for constraint in constraints:
    try:
        run_query(constraint)
    except:
        pass  # Constraint might already exist

print("✅ Database cleared and constraints recreated")

### Understanding LOAD CSV

`LOAD CSV` is powerful because it:
- Handles large files efficiently
- Supports data transformation during import
- Provides built-in error handling
- Works with local files and URLs

**Basic syntax**:
```cypher
LOAD CSV WITH HEADERS FROM 'file:///path/to/file.csv' AS row
CREATE (n:Label {property: row.column_name})
```

Let's see it in action:

In [ ]:
# Method 2: Import using LOAD CSV
print("📂 Importing data using LOAD CSV...\n")

# First, we need to get the absolute path to our CSV files
import os
current_dir = os.getcwd()
data_path = os.path.join(current_dir, 'movie_data')

print(f"📁 CSV files location: {data_path}")

# For this demo, we'll load the CSV content and pass it as parameters
# In production, you'd use actual LOAD CSV with file URLs

# Load and import genres
print("\n1️⃣ Importing genres from CSV...")
genres_df = pd.read_csv(f'{data_path}/genres.csv')

# Batch import genres (more efficient than one-by-one)
genre_data = genres_df.to_dict('records')
run_query("""
UNWIND $genres as genre
CREATE (g:Genre {name: genre.name})
""", {'genres': genre_data})

genre_count = run_query("MATCH (g:Genre) RETURN count(g) as count")[0]['count']
print(f"   ✅ Imported {genre_count} genres")

# Load and import people
print("\n2️⃣ Importing people from CSV...")
people_df = pd.read_csv(f'{data_path}/people.csv')

# Convert date strings and prepare data
people_data = []
for _, person in people_df.iterrows():
    people_data.append({
        'id': person['id'],
        'name': person['name'],
        'born': person['born'],
        'nationality': person['nationality'],
        'roles': person['roles'].split('|')  # Convert back to list
    })

run_query("""
UNWIND $people as person
CREATE (p:Person {
    id: person.id,
    name: person.name,
    born: date(person.born),
    nationality: person.nationality
})
""", {'people': people_data})

people_count = run_query("MATCH (p:Person) RETURN count(p) as count")[0]['count']
print(f"   ✅ Imported {people_count} people")

# Load and import users
print("\n3️⃣ Importing users from CSV...")
users_df = pd.read_csv(f'{data_path}/users.csv')
user_data = users_df.to_dict('records')

run_query("""
UNWIND $users as user
CREATE (u:User {
    id: user.id,
    username: user.username,
    email: user.email,
    joined: date(user.joined),
    age: user.age
})
""", {'users': user_data})

user_count = run_query("MATCH (u:User) RETURN count(u) as count")[0]['count']
print(f"   ✅ Imported {user_count} users")

# Load and import movies with genre relationships
print("\n4️⃣ Importing movies from CSV...")
movies_df = pd.read_csv(f'{data_path}/movies.csv')

# Prepare movie data
movie_data = []
for _, movie in movies_df.iterrows():
    movie_data.append({
        'id': movie['id'],
        'title': movie['title'],
        'released': movie['released'],
        'runtime': int(movie['runtime']),
        'budget': int(movie['budget']),
        'revenue': int(movie['revenue']),
        'plot': movie['plot'],
        'genres': movie['genres'].split('|')  # Convert back to list
    })

# Import movies and create genre relationships in one query
run_query("""
UNWIND $movies as movie
CREATE (m:Movie {
    id: movie.id,
    title: movie.title,
    released: date(movie.released),
    runtime: movie.runtime,
    budget: movie.budget,
    revenue: movie.revenue,
    plot: movie.plot
})
WITH m, movie
UNWIND movie.genres as genreName
MATCH (g:Genre {name: genreName})
CREATE (m)-[:IN_GENRE]->(g)
""", {'movies': movie_data})

movie_count = run_query("MATCH (m:Movie) RETURN count(m) as count")[0]['count']
genre_rel_count = run_query("MATCH ()-[r:IN_GENRE]->() RETURN count(r) as count")[0]['count']
print(f"   ✅ Imported {movie_count} movies")
print(f"   ✅ Created {genre_rel_count} genre relationships")

print("\n🎉 LOAD CSV import complete!")

### Benefits of UNWIND for Batch Operations

Notice we used `UNWIND` instead of individual CREATE statements. This is much more efficient:

**Instead of**: 50 separate queries
```cypher
CREATE (p:Person {name: 'John'});
CREATE (p:Person {name: 'Jane'});
...
```

**We use**: 1 batch query
```cypher
UNWIND $people as person
CREATE (p:Person {name: person.name})
```

This reduces network overhead and transaction costs significantly.

In [ ]:
# Let's add some sample relationships to complete our LOAD CSV demo
print("🎬 Adding sample relationships to complete the graph...\n")

# Create some ACTED_IN relationships
sample_relationships = []
movies = run_query("MATCH (m:Movie) RETURN m.id as id LIMIT 10")
people = run_query("MATCH (p:Person) RETURN p.id as id LIMIT 15")

for movie in movies:
    # Each movie gets 3-4 actors
    selected_actors = random.sample(people, 3)
    for actor in selected_actors:
        sample_relationships.append({
            'person_id': actor['id'],
            'movie_id': movie['id'],
            'role': fake.first_name() if random.random() > 0.5 else None
        })

# Batch create relationships
run_query("""
UNWIND $relationships as rel
MATCH (p:Person {id: rel.person_id}),
      (m:Movie {id: rel.movie_id})
CREATE (p)-[:ACTED_IN {role: rel.role}]->(m)
""", {'relationships': sample_relationships})

# Create some user ratings
users = run_query("MATCH (u:User) RETURN u.id as id LIMIT 5")
rating_relationships = []

for user in users:
    selected_movies = random.sample(movies, 5)
    for movie in selected_movies:
        rating_relationships.append({
            'user_id': user['id'],
            'movie_id': movie['id'],
            'rating': random.randint(1, 5),
            'timestamp': fake.date_time_between(start_date='-1y', end_date='now').isoformat()
        })

run_query("""
UNWIND $ratings as rating
MATCH (u:User {id: rating.user_id}),
      (m:Movie {id: rating.movie_id})
CREATE (u)-[:RATED {
    rating: rating.rating,
    timestamp: datetime(rating.timestamp)
}]->(m)
""", {'ratings': rating_relationships})

acted_count = run_query("MATCH ()-[r:ACTED_IN]->() RETURN count(r) as count")[0]['count']
rating_count = run_query("MATCH ()-[r:RATED]->() RETURN count(r) as count")[0]['count']

print(f"✅ Created {acted_count} ACTED_IN relationships")
print(f"✅ Created {rating_count} RATED relationships")
print("\n🎉 LOAD CSV import demonstration complete!")

## 🔧 Import Performance Tips

When working with larger datasets, these tips will save you time and resources:

In [ ]:
# Demonstrate performance optimization techniques
print("⚡ Performance optimization techniques for data import:\n")

print("1️⃣ Use PERIODIC COMMIT for large datasets:")
print("""
// For files with millions of rows
USING PERIODIC COMMIT 1000
LOAD CSV WITH HEADERS FROM 'file:///large_dataset.csv' AS row
CREATE (n:Node {property: row.column})
""")

print("\n2️⃣ Create constraints BEFORE importing:")
print("""
// Create constraints first for data integrity and performance
CREATE CONSTRAINT person_id FOR (p:Person) REQUIRE p.id IS UNIQUE;
// Then import data
""")

print("\n3️⃣ Use MERGE carefully (it's slower than CREATE):")
print("""
// MERGE is safe but slower
MERGE (p:Person {id: row.person_id})

// CREATE is faster but requires clean data
CREATE (p:Person {id: row.person_id})
""")

print("\n4️⃣ Import order matters:")
print("""
1. Create constraints
2. Import nodes (in dependency order)
3. Import relationships
4. Create additional indexes
""")

print("\n5️⃣ Use UNWIND for batch operations:")
print("""
// Batch multiple operations in one query
UNWIND $batch as item
CREATE (n:Node {property: item.value})
""")

# Let's measure the current performance of our graph
print("\n📊 Current graph performance metrics:")

# Simple query performance test
import time

start_time = time.time()
result = run_query("""
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)-[:IN_GENRE]->(g:Genre)
RETURN g.name as genre, count(*) as movie_count
ORDER BY movie_count DESC
LIMIT 5
""")
query_time = time.time() - start_time

print(f"   Query execution time: {query_time:.3f} seconds")
print(f"   Results returned: {len(result)} rows")

print("\nTop genres by number of movies:")
for row in result:
    print(f"   {row['genre']}: {row['movie_count']} movies")

print("\n✅ Performance optimization tips covered!")

## 🎯 Knowledge Check #4

**Question**: When would you use MERGE vs CREATE when importing data?

**Think about it, then check below:**

<details>
<summary>Click to see the answer</summary>

**Use CREATE when:**
- You're sure the data doesn't exist (initial load)
- You have unique constraints to prevent duplicates
- Performance is critical (CREATE is faster)
- Your data is clean and validated

**Use MERGE when:**
- Data might already exist (updates/incremental loads)
- You want upsert behavior (create if missing, match if exists)
- You're importing from multiple sources that might overlap
- Data integrity is more important than speed

**Best practice**: Use CREATE for initial loads with constraints, then MERGE for ongoing updates.

</details>

---

# 🎓 Module 2 Summary

## What You've Accomplished

Congratulations! You've completed Module 2 and learned:

### ✅ **Data Modeling Mastery**
- **Domain analysis** - Converting business requirements to graph models
- **Node and relationship design** - Choosing effective graph structures
- **Property modeling** - Balancing normalization and performance
- **Model validation** - Testing with business queries

### ✅ **Import Techniques**
- **CREATE statements** - Direct import for learning and small datasets
- **LOAD CSV** - Production-ready import for medium datasets
- **UNWIND batching** - Efficient bulk operations
- **Performance optimization** - Best practices for large-scale imports

### ✅ **Data Quality**
- **Constraints and indexes** - Ensuring data integrity and performance
- **Import validation** - Verifying successful data loads
- **Business query testing** - Confirming model effectiveness

### ✅ **Real-World Application**
- Built a complete movie recommendation system
- Handled complex relationships and properties
- Implemented collaborative filtering queries
- Demonstrated graph advantages over relational approaches

## 🚀 What's Next?

In **Module 3: Unstructured Data**, you'll learn:
- Processing text documents and extracting entities
- Building knowledge graphs from unstructured content
- Integrating NLP pipelines with Neo4j
- Creating embeddings and semantic search

## 🎯 Final Knowledge Check

Before moving on, make sure you can:

1. **Design** a graph model from business requirements
2. **Choose** appropriate import strategies for different data sizes
3. **Create** constraints and indexes for data quality
4. **Write** efficient batch import queries
5. **Validate** your model with business queries

## 💪 Practice Challenge

**Try this on your own**: Model an e-commerce system with:
- Products, Categories, Customers, Orders
- Reviews, Recommendations, Inventory
- Import data from CSV files
- Answer questions like "What do customers who bought X also buy?"

You now have all the tools you need for effective data modeling and import!

---

**🎉 Congratulations on completing Module 2! You're ready to tackle unstructured data and knowledge graphs.**

In [ ]:
# Optional: Clean up and close connection
# Uncomment these lines if you want to clean up
# run_query("MATCH (n) DETACH DELETE n")
# driver.close()

print("🎊 Module 2 Complete! Excellent work on data modeling and import!")
print("🚀 Ready for Module 3: Unstructured Data")
print("\n📊 Final graph stats:")

final_stats = run_query("""
MATCH (n)
OPTIONAL MATCH ()-[r]-()
RETURN count(DISTINCT n) as nodes, count(DISTINCT r) as relationships
""")

print(f"   Nodes: {final_stats[0]['nodes']:,}")
print(f"   Relationships: {final_stats[0]['relationships']:,}")
print("\n🎬 You've built a complete movie recommendation graph! 🌟")